In [ ]:
pip install transformers PyPDF2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
        return text

pdf_text = extract_text_from_pdf('pdf file path here')

In [ ]:
!pip install datasets
!pip install faiss-cpu

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import re

In [ ]:
def answer_question_bert(question, context, model_name="bert-large-uncased-whole-word-masking-finetuned-squad"):
    # Load tokenizer and model
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForQuestionAnswering.from_pretrained(model_name)

    # Encode the question-context pair
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512)

    # Model prediction
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Find start and end of the answer
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Convert tokens to string
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    answer = tokenizer.convert_tokens_to_string(tokens[answer_start:answer_end])

    # Extract complete sentence containing the answer
    sentences = re.findall(r"([^.]*?"+re.escape(answer)+r"[^.]*\.)", context)
    if sentences:
        return sentences[0]
    else:
        return answer

In [ ]:
question = "write your question here"
context = pdf_text
answer = answer_question_bert(question, context)
print(answer)